# Modelos para la predicción de energía electrica

1. Cargo los datos tranformados en el notebook exploración de datos.
2. Exploro las variables a utilizar en los modelos.
3. Modelo base de partida.
4. Decision Tree.
5. Random Forest.
6. Gradient Boosting Trees.
7. Comparación de modelos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
plt.style.use("seaborn")
plt.rcParams["figure.figsize"] = (18, 6)